# Quantum Fraud Detection – Clean Notebook (newfraud)
This notebook orchestrates the experiment by importing reusable code from `src/`.

In [ ]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
print('Project root:', project_root)

In [ ]:
from src.data_loader import load_csvs, merge_on_transaction_id
from src.preprocessing import PreprocessConfig, preprocess_pipeline, split_data
from src.model_classical import ClassicalConfig, train_logreg
from src.model_quantum import QuantumConfig, train_vqc
from src import evaluation as eval_mod
import yaml, os
cfg_path = os.path.join(project_root, 'configs', 'config.yaml')
cfg = yaml.safe_load(open(cfg_path, 'r'))
df_txn, df_id = load_csvs(cfg['data']['transaction_csv'], cfg['data']['identity_csv'])
df = merge_on_transaction_id(df_txn, df_id)
pp_cfg = PreprocessConfig(
    missing_threshold=cfg['preprocessing']['missing_threshold'],
    target_col=cfg['preprocessing']['target_col'],
    id_cols=cfg['preprocessing'].get('id_cols', []),
    top_k_corr_features=cfg['preprocessing']['top_k_corr_features'],
)
df_processed, selected = preprocess_pipeline(df, pp_cfg)
X_train, X_test, y_train, y_test = split_data(
    df_processed, target=pp_cfg.target_col,
    test_size=cfg['preprocessing']['test_size'],
    random_state=cfg['preprocessing']['random_state'],
    stratify=cfg['preprocessing']['stratify']
)
cl_cfg = ClassicalConfig(**cfg['classical_model'])
clf_classical = train_logreg(X_train, y_train, cl_cfg)
y_pred = clf_classical.predict(X_test)
try:
    y_proba = clf_classical.predict_proba(X_test)[:,1]
except Exception:
    y_proba = None
metrics = eval_mod.compute_metrics(y_test, y_pred, y_proba)
metrics